# Vector Ops

This tutorial will go over how to use the `--graphblas-lower` pass from `graphblas-opt` to lower some ops from the GraphBLAS dialect that deal with vectors into executable Python code. There are many such ops. We'll focus on `graphblas.vector_argmin` and `graphblas.vector_argmax`.

Since the [ops reference](../../ops_reference.rst#assumptions) already documents these ops with examples, we'll only briefly describe them here. 

Let’s first import some necessary modules and generate an instance of our JIT engine.

In [12]:
import mlir_graphblas
import numpy as np
from mlir_graphblas.tools.utils import sparsify_array, densify_vector

engine = mlir_graphblas.MlirJitEngine()

Here are the passes we'll use.

In [13]:
passes = [
    "--graphblas-lower",
    "--sparsification",
    "--sparse-tensor-conversion",
    "--linalg-bufferize",
    "--func-bufferize",
    "--tensor-bufferize",
    "--tensor-constant-bufferize",
    "--finalizing-bufferize",
    "--convert-linalg-to-loops",
    "--convert-scf-to-std",
    "--convert-memref-to-llvm",
    "--convert-std-to-llvm",
]

## Overview of graphblas.vector_argmin

Here, we'll show how to use the `graphblas.vector_argmin` op. 

`graphblas.vector_argmin` determines the [argmin](https://en.wikipedia.org/wiki/Arg_max#Arg_min) of a vector. If there are multiple values that can be the argmin, an arbitrary one is chosen from them. The given sparse vector must have rank 1 and must be sparse.

Let's create an example input sparse vector.

In [14]:
dense_vector = np.array(
    [0, 1, 0, -999, 2, 0, 3, 0],
    dtype=np.int64,
)

sparse_vector = sparsify_array(dense_vector, [True])

Here's what some MLIR code using `graphblas.vector_argmin` looks like. 

In [15]:
mlir_text = """
#CV64 = #sparse_tensor.encoding<{
  dimLevelType = [ "compressed" ],
  pointerBitWidth = 64,
  indexBitWidth = 64
}>

module {
    func @vector_argmin_wrapper(%vec: tensor<?xi64, #CV64>) -> index {
        %answer = graphblas.vector_argmin %vec : tensor<?xi64, #CV64>
        return %answer : index
    }
}
"""

Let's compile it and demonstrate its use. 

In [16]:
engine.add(mlir_text, passes)

['vector_argmin_wrapper']

In [17]:
argmin = engine.vector_argmin_wrapper(sparse_vector)
argmin

3

Let's verify that the behavior is the same as that of [NumPy](https://numpy.org/).

In [18]:
argmin == np.argmin(dense_vector)

True

## Overview of graphblas.vector_argmax

Here, we'll show how to use the `graphblas.vector_argmax` op. 

`graphblas.vector_argmax` determines the [argmax](https://en.wikipedia.org/wiki/Arg_max) of a vector. 

Here's what some MLIR code using `graphblas.vector_argmax` looks like. 

In [19]:
mlir_text = """
#CV64 = #sparse_tensor.encoding<{
  dimLevelType = [ "compressed" ],
  pointerBitWidth = 64,
  indexBitWidth = 64
}>

module {
    func @vector_argmax_wrapper(%vec: tensor<?xi64, #CV64>) -> index {
        %answer = graphblas.vector_argmax %vec : tensor<?xi64, #CV64>
        return %answer : index
    }
}
"""

Let's compile it and demonstrate its use.

In [20]:
engine.add(mlir_text, passes)

['vector_argmax_wrapper']

In [21]:
argmax = engine.vector_argmax_wrapper(sparse_vector)
argmax

6

Let's verify that the behavior is the same as that of [NumPy](https://numpy.org/).

In [22]:
argmax == np.argmax(dense_vector)

True